# Generate A big JSON : 

### Imports and Variables : 

In [3]:
import json
import random
from faker import Faker

fake = Faker()

# Create a list to store the objects
data = []
used_pricelists = set()  # Keep track of used PriceList values

# Amount of Objects
num = 122

### Create the bulk json : 


In [4]:
for _ in range(num):
    product_id = fake.uuid4().replace("-", "")  # Generate a random ProductID
    from_date = fake.date_time_this_decade().isoformat() + "Z"
    to_date = "9999-12-31T00:00:00Z"
    
    # Generate a unique random PriceList value
    while True:
        pricelist = str(random.randint(1, 500))
        if pricelist not in used_pricelists:
            used_pricelists.add(pricelist)
            break
    
    user_id = fake.bothify(text="??########")  # Randomized UserID (e.g., "AB12345678")

    obj = {
        "AllCustomers": False,
        "Vendor": None,
        "Userfield01": None,
        "LastLoginDate": "0001-01-01T00:00:00Z",
        "IsAdmin": False,
        "CanOrder": False,
        "Manager": None,
        "WeeklySummary": False,
        "RepID": user_id,
        "AddressID": 0,
        "FailedPasswordAttemptCount": 0,
        "SocialLogin": 0.0,
        "IsManager": False,
        "AllInArea": True,
        "Deleted": False,
        "Tel": None,
        "Name": f"{user_id} DRIVER",  # Randomized UserID reflected in the name
        "Role": "OFFLOAD,REPL,POD,DS,REQ,GRV",
        "DefaultBranchID": "6141",
        "Country": None,
        "MonthlyKPI": False,
        "DefaultAreaID": "ZABW00",
        "PasswordSalt": "OFFLOAD,REPL,POD,DS,REQ,GRV",
        "DailySummary": False,
        "IsLockedOut": False,
        "Email": None,
        "UserID": user_id,  # Randomized UserID
        "SupplierID": "PEPSICO-PRO",
        "AllInBranch": False,
        "AcceptTC": False,
        "DefaultWarehouseID": None,
        "SupplierIDUserID": None,
        "IsRep": False,
        "PasswordHash": "password",
        "Version": None,
        "Password": None,
        "Count": 0.0
    }
    data.append(obj)

# Save the data to a JSON file
with open("output.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("JSON file with ", num , " objects created.")

JSON file with  122  objects created.
